# Customizing Your ComputerAgent

This notebook demonstrates four practical ways to increase the capabilities and success rate of your `ComputerAgent` in the Agent SDK:

1. Simple: Prompt engineering (via optional `instructions`)
2. Easy: Tools (function tools and custom computer tools)
3. Intermediate: Callbacks
4. Expert: Custom `@register_agent` loops

> Tip: The same patterns work in scripts and services — the notebook just makes it easy to iterate.

## Setup

We'll import `ComputerAgent`, a simple Docker-based computer, and some utilities.

In [ ]:
import logging
from agent.agent import ComputerAgent
from agent.callbacks import LoggingCallback
from computer import Computer

computer = Computer(
    os_type="linux",
    provider_type="docker",
    image="trycua/cua-ubuntu:latest",
    name="my-cua-container"
)

await computer.run() # Launch & connect to Docker container

## 1) Simple: Prompt engineering

You can guide your agent with system-like `instructions`.

Under the hood, `ComputerAgent(instructions=...)` adds a `PromptInstructionsCallback` that prepends a user message before each LLM call.

This mirrors the recommended snippet in code:

```python
effective_input = full_input
if instructions:
    effective_input = [{"role": "user", "content": instructions}] + full_input
```

In [ ]:
instructions = (
    "You are a meticulous software operator. Prefer safe, deterministic actions. "
    "Always confirm via on-screen text before proceeding."
)
agent = ComputerAgent(
    model="openai/computer-use-preview",
    tools=[computer],
    instructions=instructions,
    callbacks=[LoggingCallback(level=logging.INFO)],
)
messages = [
    {"role": "user", "content": "Open the settings and turn on dark mode."}
]

# In notebooks, you may want to consume the async generator
import asyncio
async def run_once():
    async for chunk in agent.run(messages):
        # Print any assistant text outputs
        for item in chunk.get("output", []):
            if item.get("type") == "message":
                for c in item.get("content", []):
                    if c.get("text"):
                        print(c.get("text"))

await run_once()


## 2) Easy: Tools

Add function tools to expose deterministic capabilities. Tools are auto-extracted to schemas and callable by the agent.

In [ ]:
def calculate_percentage(numerator: float, denominator: float) -> str:
    """Calculate a percentage string.

    Args:
        numerator: Numerator value
        denominator: Denominator value
    Returns:
        A formatted percentage string (e.g., '75.00%').
    """
    if denominator == 0:
        return "0.00%"
    return f"{(numerator/denominator)*100:.2f}%"

agent_with_tool = ComputerAgent(
    model="openai/computer-use-preview",
    tools=[computer, calculate_percentage],
    instructions="When doing math, prefer the `calculate_percentage` tool when relevant.",
)


## 3) Intermediate: Callbacks

Callbacks offer lifecycle hooks. For example, limit recent images or record trajectories.

In [ ]:
from agent.callbacks import ImageRetentionCallback, TrajectorySaverCallback

agent_with_callbacks = ComputerAgent(
    model="anthropic/claude-3-5-sonnet-20241022",
    tools=[computer],
    callbacks=[
        ImageRetentionCallback(only_n_most_recent_images=3),
        TrajectorySaverCallback("./trajectories"),
    ],
)


## 4) Expert: Custom `@register_agent`

Register custom agent configs that implement `predict_step` (and optionally `predict_click`). This gives you full control over prompting, message shaping, and tool wiring.

See: `libs/python/agent/agent/loops/` for concrete examples.

## Next steps

- Start with `instructions` for fast wins.
- Add function tools for determinism and reliability.
- Use callbacks to manage cost, logs, and safety.
- Build custom loops for specialized domains.